In [4]:
today_date = '2026-01-05'


StatementMeta(, 81c47ea0-d42f-4148-a49c-77e1a6032d2d, 6, Finished, Available, Finished)

In [5]:
from pyspark.sql.functions import col

df = spark.read.format('delta').load('abfss://Dev_Workspace@onelake.dfs.fabric.microsoft.com/Lakehouse_sales.Lakehouse/Tables/dbo/tblsales_silver').filter(col('processing_date')==str(today_date))

StatementMeta(, 81c47ea0-d42f-4148-a49c-77e1a6032d2d, 7, Finished, Available, Finished)

In [6]:
display(df)

StatementMeta(, 81c47ea0-d42f-4148-a49c-77e1a6032d2d, 8, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 6df829d5-89fd-43c9-937a-e3d0be7fbee9)

## Creating Dimention Tables

In [7]:
#Dim_Customers Table
from pyspark.sql.types import StructType , StructField , StringType,IntegerType,DoubleType,DateType

from delta.tables import DeltaTable


#Define schemas from dimension tables

dim_customer_schema = StructType([
    StructField('Customer_ID', StringType(),True ),
    StructField('Customer_Name',StringType(),True),
    StructField('Segment',StringType(),True),
    StructField('Postal_Code',StringType(),True),
    StructField('City',StringType(),True),
    StructField('State',StringType(),True),
    StructField('Country',StringType(),True),
    StructField('Region',StringType(),True),
    StructField('Market',StringType(),True)
])


DeltaTable.createIfNotExists(spark).tableName('Dim_Customer').addColumns(dim_customer_schema).execute()

StatementMeta(, 81c47ea0-d42f-4148-a49c-77e1a6032d2d, 9, Finished, Available, Finished)

In [8]:
# Dim_Product Table
dim_product_schema = StructType([
    StructField('Product_ID',StringType(),True),
    StructField('Category',StringType(),True),
    StructField('Sub_Category',StringType(),True),
    StructField('Product_Name',StringType(),True)
])

DeltaTable.createIfNotExists(spark).tableName('Dim_Product').addColumns(dim_product_schema).execute()

StatementMeta(, 81c47ea0-d42f-4148-a49c-77e1a6032d2d, 10, Finished, Available, Finished)

## Fact Table

In [9]:
#Sales Table
Fact_Sales_schema = StructType([
    StructField('Row_ID',StringType(),True),
    StructField('Order_ID',StringType(),True),
    StructField('Customer_ID',StringType(),True),
    StructField('Product_ID',StringType(),True),
    StructField('Order_Date',DateType(),True),
    StructField('Ship_Date',DateType(),True),
    StructField('Ship_Mode',StringType(),True),
    StructField('Sales',DoubleType(),True),
    StructField('Quantity',IntegerType(),True),
    StructField('Discount',DoubleType(),True),
    StructField('Profit',DoubleType(),True),
    StructField('Shipping_Cost',DoubleType(),True),
    StructField('Order_Priority',StringType(),True),
    StructField('Month',StringType(),True),
    StructField('Year',StringType(),True),
    StructField('processing_date',StringType(),True),
    StructField('Delivery_Days',StringType(),True),
    StructField('Profit_Margin',StringType(),True)
])


DeltaTable.createIfNotExists(spark).tableName('Fact_Sales').addColumns(Fact_Sales_schema).execute()

StatementMeta(, 81c47ea0-d42f-4148-a49c-77e1a6032d2d, 11, Finished, Available, Finished)

In [10]:
df_selected_dim_customer = (
    df.select('Customer_ID',
			'Customer_Name',
			'Segment',
			'Postal_Code',
			'City',
			'State',
			'Country',
			'Region',
			'Market').dropDuplicates())

StatementMeta(, 81c47ea0-d42f-4148-a49c-77e1a6032d2d, 12, Finished, Available, Finished)

In [11]:
from delta.tables import *

dim_customer_table_path = 'abfss://Dev_Workspace@onelake.dfs.fabric.microsoft.com/Lakehouse_sales.Lakehouse/Tables/dbo/dim_customer'
dim_deltacustomer = DeltaTable.forPath(spark,dim_customer_table_path)

## Perform the MERGE (UPSERT)

dim_deltacustomer.alias('target').merge(
df_selected_dim_customer.alias('source'),'target.Customer_ID=source.Customer_ID'
).whenMatchedUpdate( set = {
			'Customer_Name':'source.Customer_Name',
			'Segment':'source.Segment',
			'Postal_Code':'source.Postal_Code',
			'City':'source.City',
			'State':'source.State',
			'Country':'source.Country',
			'Region':'source.Region',
			'Market':'source.Market'

}).whenNotMatchedInsert(values={
            'Customer_ID':'source.Customer_ID',
			'Customer_Name':'source.Customer_Name',
			'Segment':'source.Segment',
			'Postal_Code':'source.Postal_Code',
			'City':'source.City',
			'State':'source.State',
			'Country':'source.Country',
			'Region':'source.Region',
			'Market':'source.Market'

}).execute()


StatementMeta(, 81c47ea0-d42f-4148-a49c-77e1a6032d2d, 13, Finished, Available, Finished)

In [12]:
%%sql
SELECT * FROM dim_customer

StatementMeta(, 81c47ea0-d42f-4148-a49c-77e1a6032d2d, 14, Finished, Available, Finished)

<Spark SQL result set with 1000 rows and 9 fields>

In [13]:
history_df = dim_deltacustomer.history(1)  
operation_metrics = history_df.select("operationMetrics").collect()[0][0]
rows_inserted = operation_metrics.get('numTargetRowsInserted', 0)
rows_updated = operation_metrics.get('numTargetRowsUpdated', 0)
rows_deleted = operation_metrics.get('numTargetRowsDeleted', 0)
rows_affected = int(rows_inserted) + int(rows_updated) + int(rows_deleted) 

print('Total rows of table: ',dim_deltacustomer.toDF().count())
print(f"Rows inserted: {rows_inserted}")
print(f"Rows updated: {rows_updated}")
print(f"Rows deleted: {rows_deleted}")
print(f"Total rows affected: {rows_affected}")

StatementMeta(, 81c47ea0-d42f-4148-a49c-77e1a6032d2d, 15, Finished, Available, Finished)

Total rows of table:  1021
Rows inserted: 1021
Rows updated: 0
Rows deleted: 0
Total rows affected: 1021


In [14]:
df_selected_dim_product = (df.select('Product_ID',
			'Category',
			'Sub_Category',
			'Product_Name').dropDuplicates()
            )

StatementMeta(, 81c47ea0-d42f-4148-a49c-77e1a6032d2d, 16, Finished, Available, Finished)

In [15]:
from delta.tables import *

dim_product_table_path = 'abfss://Dev_Workspace@onelake.dfs.fabric.microsoft.com/Lakehouse_sales.Lakehouse/Tables/dbo/dim_product'
dim_deltaproduct = DeltaTable.forPath(spark,dim_product_table_path)

dim_deltaproduct.alias('target').merge(
    df_selected_dim_product.alias('source'),'target.Product_ID=source.Product_ID'

).whenMatchedUpdate(set = {
			'Category':'source.Category',
			'Sub_Category':'source.Sub_Category',
			'Product_Name':'source.Product_Name'

}).whenNotMatchedInsert(values={
            'Product_ID':'source.Product_ID',
			'Category':'source.Category',
			'Sub_Category':'source.Sub_Category',
			'Product_Name':'source.Product_Name'
}).execute()

StatementMeta(, 81c47ea0-d42f-4148-a49c-77e1a6032d2d, 17, Finished, Available, Finished)

In [16]:
%%sql
select * from dim_product

StatementMeta(, 81c47ea0-d42f-4148-a49c-77e1a6032d2d, 18, Finished, Available, Finished)

<Spark SQL result set with 1000 rows and 4 fields>

In [17]:
history_df = dim_deltaproduct.history(1)  
operation_metrics = history_df.select("operationMetrics").collect()[0][0]
rows_inserted = operation_metrics.get('numTargetRowsInserted', 0)
rows_updated = operation_metrics.get('numTargetRowsUpdated', 0)
rows_deleted = operation_metrics.get('numTargetRowsDeleted', 0)
rows_affected = int(rows_inserted) + int(rows_updated) + int(rows_deleted) 

print('Total rows of table: ',dim_deltaproduct.toDF().count())
print(f"Rows inserted: {rows_inserted}")
print(f"Rows updated: {rows_updated}")
print(f"Rows deleted: {rows_deleted}")
print(f"Total rows affected: {rows_affected}")

StatementMeta(, 81c47ea0-d42f-4148-a49c-77e1a6032d2d, 19, Finished, Available, Finished)

Total rows of table:  1410
Rows inserted: 1410
Rows updated: 0
Rows deleted: 0
Total rows affected: 1410


In [18]:
df_selected_fact_sales = (df.select(
            'Row_ID',
			'Order_ID',
		    'Customer_ID',
			'Product_ID',
			'Order_Date',
			'Ship_Date',
			'Ship_Mode',
			'Sales',
			'Quantity',
			'Discount',
			'Profit',
			'Shipping_Cost',
			'Order_Priority',
			'Month',
			'Year',
			'processing_date',
			'Delivery_Days',
			'Profit_Margin'
            ).dropDuplicates()
            )

StatementMeta(, 81c47ea0-d42f-4148-a49c-77e1a6032d2d, 20, Finished, Available, Finished)

In [19]:
from delta.tables import *

fact_sales_table_path = 'abfss://Dev_Workspace@onelake.dfs.fabric.microsoft.com/Lakehouse_sales.Lakehouse/Tables/dbo/fact_sales'

fact_deltasales = DeltaTable.forPath(spark,fact_sales_table_path)

fact_deltasales.alias('target').merge(
    df_selected_fact_sales.alias('source'),'target.Order_ID=source.Order_ID'

).whenMatchedUpdate(set = {
		    'Row_ID':'source.Row_ID',
			# 'Order_ID':'source.Order_ID',
		    'Customer_ID':'source.Customer_ID',
			'Product_ID':'source.Product_ID',
			'Order_Date':'source.Order_Date',
			'Ship_Date':'source.Ship_Date',
			'Ship_Mode':'source.Ship_Mode',
			'Sales':'source.Sales',
			'Quantity':'source.Quantity',
			'Discount':'source.Discount',
			'Profit':'source.Profit',
			'Shipping_Cost':'source.Shipping_Cost',
			'Order_Priority':'source.Order_Priority',
			'Month':'source.Month',
			'Year':'source.Year',
			'processing_date':'source.processing_date',
			'Delivery_Days':'source.Delivery_Days',
			'Profit_Margin':'source.Profit_Margin'
}).whenNotMatchedInsert(values={
            'Row_ID':'source.Row_ID',
			'Order_ID':'source.Order_ID',
		    'Customer_ID':'source.Customer_ID',
			'Product_ID':'source.Product_ID',
			'Order_Date':'source.Order_Date',
			'Ship_Date':'source.Ship_Date',
			'Ship_Mode':'source.Ship_Mode',
			'Sales':'source.Sales',
			'Quantity':'source.Quantity',
			'Discount':'source.Discount',
			'Profit':'source.Profit',
			'Shipping_Cost':'source.Shipping_Cost',
			'Order_Priority':'source.Order_Priority',
			'Month':'source.Month',
			'Year':'source.Year',
			'processing_date':'source.processing_date',
			'Delivery_Days':'source.Delivery_Days',
			'Profit_Margin':'source.Profit_Margin'
}).execute()

StatementMeta(, 81c47ea0-d42f-4148-a49c-77e1a6032d2d, 21, Finished, Available, Finished)

In [20]:
history_df = fact_deltasales.history(1)  
operation_metrics = history_df.select("operationMetrics").collect()[0][0]
rows_inserted = operation_metrics.get('numTargetRowsInserted', 0)
rows_updated = operation_metrics.get('numTargetRowsUpdated', 0)
rows_deleted = operation_metrics.get('numTargetRowsDeleted', 0)
rows_affected = int(rows_inserted) + int(rows_updated) + int(rows_deleted) 

print('Total rows of table: ',fact_deltasales.toDF().count())
print(f"Rows inserted: {rows_inserted}")
print(f"Rows updated: {rows_updated}")
print(f"Rows deleted: {rows_deleted}")
print(f"Total rows affected: {rows_affected}")

StatementMeta(, 81c47ea0-d42f-4148-a49c-77e1a6032d2d, 22, Finished, Available, Finished)

Total rows of table:  2018
Rows inserted: 2018
Rows updated: 0
Rows deleted: 0
Total rows affected: 2018


In [21]:
%%sql
select * from fact_sales

StatementMeta(, 81c47ea0-d42f-4148-a49c-77e1a6032d2d, 23, Finished, Available, Finished)

<Spark SQL result set with 1000 rows and 18 fields>